In [2]:
# 한글 csv 영어변환 
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time

# 크롬 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 파파고 웹사이트 열기
papago_url = "https://papago.naver.com/"
driver.get(papago_url)

# 페이지 로딩 시간 대기
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "textarea#txtSource")))

# CSV 파일 열기
file_path = r'.\crawling_data\korean81-160_final.csv'
with open(file_path, "r") as f:
    rdr = csv.reader(f)

    # 담을 빈 리스트 생성
    translated_list = []
    
    # 데이터 번역 및 출력
    for row in rdr:
        if len(row[0]) >= 3000:
            dot_index = row[0][3000:2500:-1].index('.')
            x1, x2 = row[0][:3000-dot_index+1], row[0][3000-dot_index+1:]
            input_text = x1  # 번역할 텍스트
            input_box = driver.find_element(By.CSS_SELECTOR, "textarea#txtSource")
            input_box.clear()
            input_box.send_keys(input_text)
            time.sleep(20)
    
            translate_button = driver.find_element(By.CSS_SELECTOR, "button#btnTranslate")
            translate_button.click()
    
            # 번역 결과 대기
            output_element = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div#txtTarget")))
    
            # 번역 결과 추출
            output = output_element.text
            translated_x1 = output

            input_text = x2  # 번역할 텍스트
            input_box = driver.find_element(By.CSS_SELECTOR, "textarea#txtSource")
            input_box.clear()
            input_box.send_keys(input_text)
            time.sleep(20)
    
            translate_button = driver.find_element(By.CSS_SELECTOR, "button#btnTranslate")
            translate_button.click()
    
            # 번역 결과 대기
            output_element = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div#txtTarget")))
    
            # 번역 결과 추출
            output = output_element.text
            translated_x2 = output
            sentence = translated_x1 + translated_x2

            translated_list.append(sentence)
    
            # 입력 상자 초기화
            input_box.clear()

        else:
            input_text = row[0]  # 번역할 텍스트
            input_box = driver.find_element(By.CSS_SELECTOR, "textarea#txtSource")
            input_box.clear()
            input_box.send_keys(input_text)
            time.sleep(20)
    
            translate_button = driver.find_element(By.CSS_SELECTOR, "button#btnTranslate")
            translate_button.click()
    
            # 번역 결과 대기
            output_element = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div#txtTarget")))
    
            # 번역 결과 추출
            output = output_element.text
            translated_list.append(output)
    
            # 입력 상자 초기화
            input_box.clear()

# 드라이버 종료
driver.quit()

In [3]:
import csv
with open('translated81-160.csv', 'a', newline='') as f:
    writer  = csv.writer(f)
    for essay in translated_list:
        writer.writerow([essay])